# Wikipedia Retrieval with Claude

This notebook provides a step-by-step guide for using the Wikipedia search tool with Claude. We will:

1. Set up the environment and imports
2. Build a search tool to query the Wikipedia API
3. Test the search tool  
4. Create a Claude client with access to the tool 
5. Compare Claude's responses with and without access to the tool

## Imports and Configuration 

First we'll import libraries and load environment variables. This includes setting up logging so we can monitor the process.

In [1]:
import os
import sys
import dotenv
import anthropic

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import claude_retriever

# Load environment variables
dotenv.load_dotenv()

True

In [2]:
# Import and configure logging 
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to log to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

## Make a Wikipedia Search Tool

After importing the WikipediaSearchTool class, we can easily initalize a new WikipediaSearchTool object.

Let's test the search tool to make sure it works:

In [3]:
from claude_retriever.searcher.searchtools.wikipedia import WikipediaSearchTool

# Create a searcher
wikipedia_search_tool = WikipediaSearchTool()

# Run a test query
query = "LK-99"

results = wikipedia_search_tool.search(query, n_search_results_to_use=1)
print(results)


<search_results>
<item index="1">
<page_content>
Page Title: Room-temperature superconductor
Page Content:
A room-temperature superconductor is a material capable of displaying superconductivity at temperatures above 0 °C (273 K; 32 °F), which are commonly encountered in everyday settings. As of 2023, the material with the highest accepted superconducting temperature was highly pressurized lanthanum decahydride, whose transition temperature is approximately 250 K (−23 °C) at 200 GPa.At standard atmospheric pressure, cuprates currently hold the temperature record, manifesting superconductivity at temperatures as high as 138 K (−135 °C). Over time, researchers have consistently encountered superconductivity at temperatures previously considered unexpected or impossible, challenging the notion that achieving superconductivity at room temperature was unfeasible. The concept of "near-room temperature" transient effects has been a subject of discussion since the early 1950s.


== Significan

## Use Claude with Retrieval

Now we simply pass the search tool to a Claude client, allowing Claude access to use it when generating responses. 

In [4]:
ANTHROPIC_SEARCH_MODEL = "claude-2"

client = claude_retriever.ClientWithRetrieval(api_key=os.environ['ANTHROPIC_API_KEY'], verbose=True, search_tool = wikipedia_search_tool)

query = "Can you explain what LK-99 is?"
prompt = f'{anthropic.HUMAN_PROMPT} {query}{anthropic.AI_PROMPT}'

Here is the basic response to the query (no access to the tool).

In [5]:
basic_response = client.completions.create(
    prompt=prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)
print('-'*50)
print('Basic response:')
print(prompt + basic_response.completion)
print('-'*50)

2023-08-28 17:06:21,573 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
--------------------------------------------------
Basic response:


Human: Can you explain what LK-99 is?

Assistant: Unfortunately I do not have enough context to fully explain what "LK-99" refers to specifically. LK-99 could potentially refer to:

- A model number or product code for some type of technology product. Companies often use combinations of letters and numbers to identify specific models or SKUs.

- A specific chemical compound or molecule used in scientific research. Researchers sometimes use alphanumerical codes to refer to compounds.

- A location code or identifier, like a license plate number, product serial number, etc. 

- A coded reference to something classified or confidential. Without more context I can't determine what LK-99 denotes in this particular case.

To provide a more detailed explanation, I would need additional context about where you s


Now let's try getting a completion from Claude equipped with the Wikipedia search tool.

In [6]:
augmented_response = client.completion_with_retrieval(
    query=query,
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=1,
    max_searches_to_try=3,
    max_tokens_to_sample=1000)

print('-'*50)
print('Augmented response:')
print(prompt + augmented_response)
print('-'*50)

2023-08-28 17:06:26,354 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2023-08-28 17:06:26,355 - claude_retriever.client - INFO -  <thinking>
To answer what LK-99 is, I likely need to gather basic information on what it is, what it is used for, and any key details about its history or development. I should look for general overview information first before getting into specifics.
</thinking>

<search_query>LK-99
2023-08-28 17:06:26,355 - claude_retriever.client - INFO - Attempting search number 0.
2023-08-28 17:06:26,356 - claude_retriever.client - INFO - 
--------------------
Pausing stream because Claude has issued a query in <search_query> tags: <search_query>LK-99</search_query>
--------------------
2023-08-28 17:06:26,356 - claude_retriever.client - INFO - Running search query against SearchTool: LK-99
2023-08-28 17:06:26,801 - claude_retriever.client - INFO - 
--------------------
The SearchTool has returned the following search result